In [ ]:
import gensim
from gensim.models import Word2Vec
import re
import pandas as pd
import numpy as np
from scipy import linalg
import pickle
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [2]:
stop_words = []
with open('stopwords', 'rb') as fp:
    stop_words = pickle.load(fp)

In [44]:
cleaned_tags = []

In [45]:
 with open('data/artist_tags.csv', 'r') as inpFile:
        lines = inpFile.readlines()
        for line in lines:
            if line.isspace() is False:
                tags = tokenizer.tokenize(line)
                final_tags = []
                if len(tags) > 0:
                    for tag in tags:
                        if len(tag) > 1 and tag not in stop_words:
                            final_tags.append(tag.lower())
                    if len(final_tags) > 0:
                        cleaned_tags.append(list(set(final_tags)))

In [46]:
len(cleaned_tags)

99450

In [ ]:
len(cleaned_tags_2)

In [48]:
cleaned_tags[51961]

['punk',
 'steam',
 'surreal',
 'victorian',
 'black',
 'collage',
 'sepia',
 'steampunk',
 'vintage',
 'cephalopod',
 'gear',
 'octopus',
 'tentacles',
 'lovecraft',
 'cool',
 'cthulhu',
 'gentleman',
 'character',
 'squid',
 'animal',
 'digital']

In [18]:
word2vec_sg = Word2Vec(sentences=cleaned_tags, size=100, window=5, min_count=1, workers=4, sg=1, iter=10)

In [19]:
word2vec_sg.save("word2vec_model")

In [21]:
w2v = Word2Vec.load("word2vec_model")

In [22]:
w2v.most_similar(positive=['arctic', 'mardy', 'oriques'])

[('onelinedrawing', 0.927032470703125),
 ('thenbh', 0.9215524792671204),
 ('antemasque', 0.9204072952270508),
 ('wpsiatwin', 0.9184987545013428),
 ('understatement', 0.9125391244888306),
 ('altrock', 0.9112753868103027),
 ('brianstorm', 0.9094448089599609),
 ('no1', 0.9080705642700195),
 ('egipt', 0.9079245328903198),
 ('hampstead', 0.9073182940483093)]

In [23]:
w2v.most_similar(positive=['linkin', 'awards'])

[('antionette', 0.8257869482040405),
 ('bastilledan', 0.821168065071106),
 ('jese', 0.8099668622016907),
 ('subteranian', 0.809721827507019),
 ('kigns', 0.8039277791976929),
 ('jme', 0.8032917380332947),
 ('kygo', 0.8004037737846375),
 ('parting', 0.7984482049942017),
 ('feelin', 0.7982733249664307),
 ('nbh', 0.7981569170951843)]

##### Now we can predict a tag given other tags

##### Given a search query, lets try and find the right products

In [24]:
a = []
a.append(w2v['pikachu'])

In [25]:
b = w2v['pokemon']

In [26]:
np.dot(a, b)/linalg.norm(a)/linalg.norm(b)

array([ 0.91971552], dtype=float32)

In [49]:
product_data = []

In [50]:
for key,product in enumerate(cleaned_tags):
    if len(product) == 0:
        print(product, key)
    feature_vector = np.zeros((100,), dtype="float32")
    nwords = len(product)
    for token in product:
        feature_vector = np.add(feature_vector, w2v[token])
    
    feature_vector = np.divide(feature_vector, nwords)
    product_data.append({'tags':product, 'vector':feature_vector})

In [51]:
cleaned_tags[51961]

['punk',
 'steam',
 'surreal',
 'victorian',
 'black',
 'collage',
 'sepia',
 'steampunk',
 'vintage',
 'cephalopod',
 'gear',
 'octopus',
 'tentacles',
 'lovecraft',
 'cool',
 'cthulhu',
 'gentleman',
 'character',
 'squid',
 'animal',
 'digital']

In [52]:
product_vector_df = pd.DataFrame(product_data,columns=['tags', 'vector'])


In [53]:
product_vector_df.head()

,tags,vector
0,"[team, grunt, rocket, pokemon, uniform, villain]","[0.0550501, 0.0269306, 0.327811, -0.56768, 0.0..."
1,"[toriyama, dbz, dub, 4kids, infinite, ball, ak...","[0.142125, 0.114305, 0.505677, 0.102514, 0.023..."
2,"[davinci, cute, tv, geek, cartoons, pikachu, p...","[-0.00838749, 0.0472564, 0.324731, -0.293568, ..."
3,"[red, evolution, starter, fire, pokemon]","[-0.00421771, 0.251726, 0.239894, -0.423287, 0..."
4,"[cute, funny, nightfury, train, geek, cool, ve...","[0.216137, -0.0235948, 0.450602, -0.337562, -0..."


In [54]:
product_vector_df.shape

(99450, 2)

In [56]:
'''
search_query_tags = []
with open('user_search_keywords.tsv') as searchFile:
    searches = searchFile.readlines()
    for search in searches[1:]:
        if search.isspace() is False:
            tags = search.split("\t")[1]
            tags = tags.replace("\n", "").replace("\"", "")
            tags = tags.split(",")
            b = []
            [generate_n_gram_tokens(tag) for tag in tags]
            search_query_tags.append(list(set(b)))
            #print(search_query_tags)
'''

'\nsearch_query_tags = []\nwith open(\'user_search_keywords.tsv\') as searchFile:\n    searches = searchFile.readlines()\n    for search in searches[1:]:\n        if search.isspace() is False:\n            tags = search.split("\t")[1]\n            tags = tags.replace("\n", "").replace(""", "")\n            tags = tags.split(",")\n            b = []\n            [generate_n_gram_tokens(tag) for tag in tags]\n            search_query_tags.append(list(set(b)))\n            #print(search_query_tags)\n'

In [55]:
words_in_vocab = set(w2v.index2word)

In [61]:
def get_cosine_similarity(row, search_vector):
    try:
        cosine_similarity = np.dot(row['vector'], search_vector)/linalg.norm(row['vector'])/linalg.norm(search_vector)
        return cosine_similarity
    except Exception as e:
        raise e

In [71]:
search_query = ['hello', 'cat', 'dog', 'monkey', 'dbz' , 'dragon' , 'goku']

In [72]:
    search_vector = np.zeros((100,), dtype="float32")
    for tag in search_query:
        if tag in words_in_vocab:
            search_vector = np.add(search_vector, w2v[tag])
    product_vector_df['cosine_similarity'] = product_vector_df.apply(get_cosine_similarity, axis=1, 
                                                                     args=(search_vector,))               

In [74]:
df = product_vector_df.sort(columns='cosine_similarity', ascending=False)[:10]

/Users/nischalhp/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [76]:
df.head()

,tags,vector,cosine_similarity
11288,"[dbz, silhouette, monkey, super, negative, gok...","[0.187375, 0.0217866, 0.501207, 0.0109512, 0.0...",0.877507
710,"[grandpa, dbz, toriyama, nostalgic, gt, goku, ...","[0.349225, 0.0458218, 0.541412, 0.00121609, -0...",0.873349
45321,"[toriyama, dbz, design, amazing, gt, goku, say...","[0.176304, 0.0117984, 0.536176, 0.0928079, -0....",0.870478
681,"[gt, cloud, dragonball, goku, pikachu, pokemon...","[0.162702, 0.0341797, 0.831281, 0.118771, 0.27...",0.864560
3570,"[kameha, dbz, power, 9000, dragon, goku, anime...","[0.180594, 0.0332023, 0.425862, 0.124274, 0.18...",0.863071


In [80]:
df[['tags','cosine_similarity']].to_json(orient="records")

'[{"tags":["dbz","silhouette","monkey","super","negative","goku","vegetal","manga","space","anime","dragon","saiyan","ball","cartoon","gohan","moon"],"cosine_similarity":0.877506697},{"tags":["grandpa","dbz","toriyama","nostalgic","gt","goku","anime","dragon","ball","gohan","comics","manga","fanart","dragonballz","dragonball","akira","db","son","paper","grandson","kid","baby","nostalgia"],"cosine_similarity":0.8733491791},{"tags":["toriyama","dbz","design","amazing","gt","goku","sayain","art","anime","trunks","dragon","pokemon","ball","gohan","shirt","tee","ssj2","series","television","circle","goten","vageta","super","akira","cartoon","db","tv","awesome","corallo","drawing"],"cosine_similarity":0.8704779067},{"tags":["gt","cloud","dragonball","goku","pikachu","pokemon","dragon","ball"],"cosine_similarity":0.8645601717},{"tags":["kameha","dbz","power","9000","dragon","goku","anime","sayian","ball","crimson","west"],"cosine_similarity":0.8630713196},{"tags":["dbz","dragon","goku","kid",